# Two federal programs at odds
## School lunch debt in Durham Public Schools and competition between CEP and Title I

In [1]:
# Load packages
require(ggplot2)
require(dplyr)
require(tidyr)
require(knitr)
require(stringr)

# Do not read strings as factors
# No scientific notation
options(stringsAsFactors = FALSE, scipen = 999)

# Load lunch debt and 2018-19 banding data
debt = read.csv('./data/meal_debt.csv')
bands = read.csv('./data/banding_1718.csv')

# Load CEP eligibility data
# File names as list
cep.list = list.files('./data') %>%
    grep('cep', ., value=TRUE) %>%
    paste('./data/', ., sep='')

# Read files
cep.dat = lapply(cep.list, function(x) read.csv(x))

# Add year
year = c('2014-15', '2015-16', '2016-17', '2017-18', '2018-19')
for(i in 1:length(cep.dat)) {
  cep.dat[[i]]$year = year[i]
}
                 
# Merge to one dataset
cep = do.call(plyr::rbind.fill, cep.dat)
                 
# Filter to Durham
cep = cep %>%
    filter(lea_no == 320)

# Add missing school numbers for 2014-15 CEP eligibility data
school_no_key = cep %>%
  filter(!is.na(school_no)) %>%
  select(school_name, school_no) %>%
  unique()
school_key = school_no_key$school_no
names(school_key) = school_no_key$school_name  

cep$school_no[cep$year=='2014-15'] = school_key[cep$school_name[cep$year=='2014-15']] %>% unlist() %>% as.vector()           

# Control for data types
cep$school_no = cep$school_no %>% as.character()
debt$school_no = debt$school_no %>% as.character()

# Indicator for CEP eligibility, participation
cep$eligible[grep('X', cep$eligible)] = 1
cep$eligible[cep$eligible!=1] = 0
cep$eligible = cep$eligible %>% as.numeric()
                 
cep$participating[grep('X', cep$participating)] = 1
cep$participating[cep$participating!=1] = 0
cep$participating = cep$participating %>% as.numeric()

# Remove unnecessary columns
cep = cep %>% subset(., select=-c(lea_name, lea_no))

# Control for conflicting school names                 
debt = debt %>% subset(., select=-c(school_name))
                 
# Merge lunch debt and CEP data
df = full_join(debt, cep, by = c('school_no', 'year'))
                 
# Fill missing names from CEP data
school_key2 = names(school_key)
names(school_key2) = school_no_key$school_no
df$school_name[is.na(df$school_name)] = school_key2[df$school_no[is.na(df$school_name)]] %>% unlist() %>% as.vector()
                 
# remove DPS hospital school since non-standard school
df = df[-which(df$school_name=="DPS Hospital School"),]

# remove 389, 700 since no longer active
# remove 353, housed in Durham Tech
df = df[-which(df$school_no %in% c(389, 700, 353)),]

Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: tidyr
Loading required package: knitr
Loading required package: stringr


### School lunch debt and CEP

In [2]:
df %>%
    group_by(year) %>%
    summarise(debt = sum(unpaid))

year,debt
<chr>,<dbl>
2010-11,204692.22
2011-12,112706.47
2012-13,117526.02
2013-14,108231.05
2014-15,85092.75
2015-16,78428.06
2016-17,127939.71
2017-18,209021.50
2018-19,NA


Last school year, Durham public schools had just over $209,000 in school lunch debt--the most the district has seen in the past eight years.

In [3]:
df %>%
    group_by(year) %>%
    summarise(eligible = sum(eligible, na.rm = TRUE),
             paricipating = sum(participating, na.rm = TRUE))

year,eligible,paricipating
<chr>,<dbl>,<dbl>
2010-11,0,0
2011-12,0,0
2012-13,0,0
2013-14,0,0
2014-15,25,0
2015-16,27,10
2016-17,28,12
2017-18,25,13
2018-19,30,13


Last year, 12 of the 25 schools eligible for CEP did not participate in the program.

In [4]:
# What percent of schools are eligible and not participating in CEP?
n = df %>%
    filter(year=='2017-18') %>%
    nrow()
12/n

# Percent of debt
enp_debt = df %>%
    filter(year=='2017-18',
          eligible==1,
          participating==0) %>%
    select(unpaid) %>%
    sum(.)

enp_debt

debt_1718 = sum(df$unpaid[df$year=='2017-18'])
enp_debt/debt_1718

[1] 0.2352941

[1] 79671.11

[1] 0.3811623

Last year, 24 percent of schools account for just under 40 percent of all unpaid meals. They were all eligible for, but not participating in CEP.

### CEP and Title I

What are the 2018-19 Title I allocation bands for Durham?

In [9]:
bands %>%
    group_by(allocation_per_pupil) %>%
    summarise(lower = min(percent_low_income),
             upper = max(percent_low_income))

allocation_per_pupil,lower,upper
<dbl>,<dbl>,<dbl>
440.54,0.3882,0.5723
450.00,0.6196,0.7358
464.00,0.8297,0.8583
477.00,0.8874,1.0000


How would Title I allocations in Durham Public Schools change if all schools eligible for CEP participated?

In [5]:
# Calculate ISP with multiplier
df$isp_mult = df$isp*1.6

cep$school_no = cep$school_no %>% as.numeric()
bands$school_no = bands$school_no %>% as.numeric()

# Save rank before
bands = bands %>% 
    arrange(-percent_low_income)
bands$rank_before = seq(1,nrow(bands))

# Index of eligible, non-participating CEP schools for 2018-19 and ISP*1.6 percentages
cep_enp = df %>%
    filter(year=='2018-19',
          eligible==1,
          participating==0) %>%
    select(school_no, isp_mult) %>%
    as.list()
names(cep_enp$isp_mult) = cep_enp$school_no

# Replace percentages
bands$percent_with_cep = bands$percent_low_income
ix = which(bands$school_no %in% cep_enp$school_no) # Index
toReplace = bands$school_no[ix] # School number
newVal = cep_enp$isp_mult[as.character(toReplace)] %>% unlist() %>% as.vector() # Value to replace
bands$percent_with_cep[ix] = newVal # Replace

# Calculate new rankings
bands = bands %>% 
    arrange(-percent_with_cep)
bands$rank_after = seq(1,nrow(bands))

# How many schools shifted ranks?
bands %>%
    filter(rank_before != rank_after) %>%
    select(rank_before, rank_after, school_name, percent_low_income, percent_with_cep)

rank_before,rank_after,school_name,percent_low_income,percent_with_cep
<int>,<int>,<chr>,<dbl>,<dbl>
14,9,James E Shepard Middle,0.8425,0.9440
9,10,Merrick-Moore Elementary,0.9159,0.9159
10,12,Burton Elementary,0.8874,0.8874
12,13,R N Harris Elementary,0.8520,0.8520
15,14,Brogden Middle,0.8416,0.8480
21,15,Parkwood Elementary,0.6713,0.8480
13,16,Neal Middle,0.8450,0.8320
16,17,Bethesda Elementary,0.8300,0.8300
23,19,Sandy Ridge Elementary,0.6545,0.7840


If every school eligible for CEP had participated, 16 schools would have shifted in the ranks used to determine Title I funding.

In [6]:
# Estimate changes in funding
# First, find number of schools in each band
banding = bands %>%
    group_by(allocation_per_pupil) %>%
    summarise(n = n())
banding

allocation_per_pupil,n
<dbl>,<int>
440.54,5
450.00,8
464.00,7
477.00,10


There are 10 schools in the highest band, 7 in the next, then 8, then 5.

In [7]:
# Bands already sorted by CEP 
# Change allocations based on rank, assuming same n and amount
bands$allocation_with_cep = bands$allocation_per_pupil
bands$allocation_with_cep[1:10] = 477
bands$allocation_with_cep[11:17] = 464
bands$allocation_with_cep[18:25] = 450
bands$allocation_with_cep[26:nrow(bands)] = 440.54

# Estimate total allocation change 
# First calculate new n low income students based on new percentage
n_with_cep = ceiling((bands$n_low_income/bands$percent_low_income)*bands$percent_with_cep)
bands$estimated_allocation = bands$allocation_with_cep*n_with_cep

# Calculate estimated in allocations
bands %>%
    group_by(school_name) %>%
    summarise(change_in_funding = round(estimated_allocation - actual_allocation,2)) %>%
    arrange(change_in_funding)

school_name,change_in_funding
<chr>,<dbl>
Holt Elementary,-31830.00
Neal Middle,-4640.00
Forest View Elementary,-4436.74
Burton Elementary,-4303.00
Bethesda Elementary,0.00
C C Spaulding Elementary,0.00
Club Boulevard Elementary,0.00
Creekside Elementary,0.00
E K Powe Elementary,0.00


According to our analysis, most schools would see no change in Title I funding had all eligible CEP schools participated. More specifically, our analysis estimates that three schools would lose just under \$5,000 while Holt Elementary could lose over \$30,000 in Title I funding.